## Homework 9

*Jackson Rudoff*

**November 15, 2022**

Read in the packages I'll need...

In [37]:
import pandas as pd
import lxml
import requests
import re
from bs4 import BeautifulSoup


### Question 1

For this one, I'm gonna just use the built-in Pandas function.

In [2]:
df_html = pd.read_html('https://en.wikipedia.org/wiki/Mitch_McConnell')

df_html



[                    Mitch McConnell  \
 0           Official portrait, 2016   
 1                               NaN   
 2            Senate Minority Leader   
 3                         Incumbent   
 4   Assumed office January 20, 2021   
 ..                              ...   
 56                       Allegiance   
 57                   Branch/service   
 58                 Years of service   
 59                             Unit   
 60                              NaN   
 
                                     Mitch McConnell.1  
 0                             Official portrait, 2016  
 1                                                 NaN  
 2                              Senate Minority Leader  
 3                                           Incumbent  
 4                     Assumed office January 20, 2021  
 ..                                                ...  
 56                                      United States  
 57                                 United States Army  
 58  

### Question 2

Here I'm gonna use Beautiful Soup to get Mitch McConnell's entire Wikipedia page. 

In [50]:
response = requests.get('https://en.wikipedia.org/wiki/Mitch_McConnell')
parsed = BeautifulSoup(response.content, 'html.parser')
parsed.title #show title

<title>Mitch McConnell - Wikipedia</title>

And now to get the table. According to the inspect, we want something called "infobox vcard."

In [53]:
get_table = parsed.find(class_= "infobox vcard")
get_table.text

'Mitch McConnellOfficial portrait, 2016Senate Minority LeaderIncumbentAssumed office January 20, 2021WhipJohn ThunePreceded byChuck SchumerIn officeJanuary 3, 2007\xa0– January 3, 2015WhipTrent LottJon KylJohn CornynPreceded byHarry ReidSucceeded byHarry ReidUnited States Senatorfrom KentuckyIncumbentAssumed office January 3, 1985Serving with Rand PaulPreceded byWalter Dee HuddlestonLeader of the Senate Republican ConferenceIncumbentAssumed office January 3, 2007Preceded byBill FristSenate Majority LeaderIn officeJanuary 3, 2015\xa0– January 20, 2021WhipJohn CornynJohn ThunePreceded byHarry ReidSucceeded byChuck SchumerSenate Majority WhipIn officeJanuary 3, 2003\xa0– January 3, 2007LeaderBill FristPreceded byHarry ReidSucceeded byDick DurbinChairman of the Senate Rules CommitteeIn officeJanuary 20, 2001\xa0– June 6, 2001Preceded byChris DoddSucceeded byChris DoddIn officeJanuary 3, 1999\xa0– January 3, 2001Preceded byJohn WarnerSucceeded byChris DoddJudge/Executive of Jefferson County

### Question 3

Now we're getting a bit more complex. I'm doing *bit* of wonyk coding here to address the fact that the name we want is at the very top of the page. But we will be able to resolve it when we process the data with Pandas. 

In [5]:
all_table_headers = [i.text for i in get_table.find_all(class_ = ["infobox-above","infobox-label"])]
all_table_headers[0] #so the top of our returned data is the full name

'Mitch McConnell'

In [6]:
all_table_info = [i.text for i in get_table.find_all(class_ = ['fn', 'infobox-data'])]
all_table_info

['Mitch McConnell',
 'John Thune',
 'Chuck Schumer',
 'Trent LottJon KylJohn Cornyn',
 'Harry Reid',
 'Harry Reid',
 'Walter Dee Huddleston',
 'Bill Frist',
 'John CornynJohn Thune',
 'Harry Reid',
 'Chuck Schumer',
 'Bill Frist',
 'Harry Reid',
 'Dick Durbin',
 'Chris Dodd',
 'Chris Dodd',
 'John Warner',
 'Chris Dodd',
 'Todd Hollenbach III',
 'Bremer Ehrler',
 'Gerald Ford',
 'Vincent Rakestraw',
 'Michael Uhlmann',
 'Addison Mitchell McConnell III (1942-02-20) February 20, 1942 (age\xa080)Sheffield, Alabama, U.S.',
 'Republican',
 '\n\nSherrill Redmon\n\u200b \u200b(m.\xa01968; div.\xa01980)\u200b\n\nElaine Chao \u200b(m.\xa01993)\u200b\n',
 '3',
 'Louisville, Kentucky, U.S.',
 '\nUniversity of Louisville (BA)\nUniversity of Kentucky (JD)\n',
 '',
 'Senate website',
 '\xa0United States',
 '\xa0United States Army',
 'July 9, 1967, to August 15, 1967 (37 days) (medical separation)',
 'United States Army Reserve']

In [54]:
mitch_df = pd.DataFrame({'key': all_table_headers, 'value': all_table_info})
mitch_df['key'][mitch_df['key'] == mitch_df['value']] = 'Full Name' #fix the first row
mitch_df = mitch_df[mitch_df['key'].isin(['Full Name','Children','Political party'])].reset_index(drop=True)


mitch_df

,key,value
0,Full Name,Mitch McConnell
1,Political party,Republican
2,Children,3


In [55]:
mitch_df.iloc[-1:]

,key,value
2,Children,3


## Question 4

Back-tracking here, we just have to take all these chunks and put it in a function. The only requirement is that you input a string (which is also forced because Python cannot process URLs without quotes). I've added code that deals with cases like Susan Collins who have no children. If 'Children' is not present in the column 'keys', then it will add a row with 'Children' and '0'. 

In [61]:
def get_wiki_info(page):
    
    assert type(page) == str, "Please input a valid URL string."

    response = requests.get(page)
    page = response.content
    parsed = BeautifulSoup(response.content, 'html.parser')

    get_table = parsed.find(class_= "infobox vcard")
    all_table_headers = [i.text for i in get_table.find_all(class_ = ["infobox-above","infobox-label"])]
    all_table_info = [i.text for i in get_table.find_all(class_ = ['fn', 'infobox-data'])]

    output_df = pd.DataFrame({'key': all_table_headers, 'value': all_table_info})
    output_df['key'][output_df['key'] == output_df['value']] = 'Full Name'
    output_df = output_df[output_df['key'].isin(['Full Name','Children','Political party'])].reset_index(drop=True)

    if ('Children' in output_df.key.values) == False:
        output_df.loc[len(output_df.index)] = ['Children', 0]
    
    return output_df
    

In [63]:
get_wiki_info('https://en.wikipedia.org/wiki/Tammy_Duckworth')

,key,value
0,Full Name,Tammy Duckworth
1,Political party,Democratic
2,Children,2


In [64]:
get_wiki_info('https://en.wikipedia.org/wiki/Susan_Collins')

,key,value
0,Full Name,Susan Collins
1,Political party,Republican
2,Children,0


In [67]:
## What if you give the wrong input type?

get_wiki_info(77)

AssertionError: Please input a valid URL string.

### Bonus: Get All Senators

First, we have to extract the links from the page. This is actually a bit tricky because just copying the selector will not work, so you kind of have to work backwards until you get to the correct child path. 

In [65]:
response_senators = requests.get('https://en.wikipedia.org/wiki/List_of_current_United_States_senators')
page_senators = response_senators.content
parsed_senators = BeautifulSoup(response_senators.content, 'html.parser')
get_links = [i.get('href') for i in parsed_senators.select('th > span > span > span > a')]

get_links[0:5]


['/wiki/Richard_Shelby',
 '/wiki/Tommy_Tuberville',
 '/wiki/Lisa_Murkowski',
 '/wiki/Dan_Sullivan_(U.S._senator)',
 '/wiki/Kyrsten_Sinema']

I want these to be actual links:

In [66]:
wiki = 'https://en.wikipedia.org'
sen_links = [wiki + link for link in get_links]

sen_links[0:10]

['https://en.wikipedia.org/wiki/Richard_Shelby',
 'https://en.wikipedia.org/wiki/Tommy_Tuberville',
 'https://en.wikipedia.org/wiki/Lisa_Murkowski',
 'https://en.wikipedia.org/wiki/Dan_Sullivan_(U.S._senator)',
 'https://en.wikipedia.org/wiki/Kyrsten_Sinema',
 'https://en.wikipedia.org/wiki/Mark_Kelly',
 'https://en.wikipedia.org/wiki/John_Boozman',
 'https://en.wikipedia.org/wiki/Tom_Cotton',
 'https://en.wikipedia.org/wiki/Dianne_Feinstein',
 'https://en.wikipedia.org/wiki/Alex_Padilla']

There are some transformations here that get a bit wonky, so I apologize if the code is a bit confusing at times. I've commented in-line for the parts that involve a cleaning process.

In [72]:
def get_all_senators(page):
    assert type(page) == str, "Please input a valid URL string."

    response = requests.get(page)
    page = response.content
    parsed = BeautifulSoup(response.content, 'html.parser')

    get_table = parsed.find(class_= "infobox vcard")
    all_table_headers = [i.text for i in get_table.find_all(class_ = ["infobox-above","infobox-label"])]
    all_table_info = [i.text for i in get_table.find_all(class_ = ['fn', 'infobox-data'])]

    output_df = pd.DataFrame({'Full Name': all_table_headers[0], 'Party': all_table_info[all_table_headers.index('Political party')]}, index=[0])
    if ('Children' in all_table_headers) == True:
        output_df['Children'] = all_table_info[all_table_headers.index('Children')]
    else:
        output_df['Children'] = 0
 
    output_df['Party'] = output_df['Party'].str.extract(r'(Democrat|Republican|Independent)') #get only parties, not longer strings
    output_df['Children'] = output_df['Children'].astype(str).str.extract(r'(\d)') #get only children's ages
    output_df.loc[output_df['Full Name'] == 'Dianne Feinstein', ['Children']] = 1 #for some reason whoever did Feinstein's page NAMED her child instead of giving a number
    output_df['Children'] = output_df['Children'].astype(int) #make numeric
    
    
    return output_df

Putting it all together:

In [73]:
chunks =[]
for link in sen_links:
    tmp = get_all_senators(link)
    chunks.append(tmp)

final_df = pd.concat(chunks).reset_index()

In [ ]:
### Avg. Number of Children

avg_chil = final_df.Children.mean().__round__() #really 2.34 but I don't think you can divide children into thirds...
print(f'The average Senator has {avg_chil} children.')

The average Senator has 2 children.


In [35]:
### Avg. Number by Party
pd.DataFrame(final_df.groupby('Party')['Children'].mean())

,Children
Party,
Democrat,1.916667
Independent,3.000000
Republican,2.720000
